### Вернуть словарь аудиозаписей VK

In [3]:
from bs4 import BeautifulSoup as bs

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

def audios(url):
    driver = webdriver.Firefox(
                            firefox_binary=FirefoxBinary('C:\Program Files\\Firefox Developer Edition\\firefox.exe'),
                            executable_path='C:\\Users\\xenia\\Documents\\geckodriver.exe',
                            options=Options().add_argument("--headless"))
    driver.get(url)
    return driver.page_source

html = audios('https://vk.com/audios112435967')

In [27]:
soup = bs(html, 'html.parser')

### Случайное сообщение с smssend

In [4]:
import requests
from random import randint

class Soup:
    def __init__(self, url, param_name):
        self.url = url
        self.param_name = param_name
        self.page_count = self.calculate_pages()
    
    def make_soup(url, params):
        req = requests.get(url, params=params,
                          headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh;' \
            'Intel Mac OS X 10.9; rv:45.0) ' \
            'Gecko/20100101 Firefox/45.0'
        })
        return bs(req.text, 'html.parser')
    
    def calculate_pages(self):
        soup = Soup.make_soup(self.url, {self.param_name: 1})
        links = soup.find_all('a')
        end_n = int(links[6]['href'].split('=')[-1])
        return end_n
    
    def random_page(self) -> list:
        divs = Soup.make_soup(
            self.url,
            {self.param_name: randint(1, self.page_count)}
        ).find_all('div', class_='center-col__content-send_mess')
        #[(to, text),...]
        return [(d.contents[1].text, d.contents[3].text) for d in divs]
    
    def random_quote(self):
        rnd_page = self.random_page()
        return rnd_page[randint(0, len(rnd_page))]

In [5]:
soup = Soup(
    url='https://smsend.ru/sent-sms.php',
    param_name='PAGEN_1',
)
soup.random_quote()

('+7 919 742 ... 23', 'Зачем ты так с моей сестрой,она ведь тебе преданна')

In [48]:
import telebot

bot = telebot.TeleBot('1314914901:AAHqioOQefwJREFoUp7N8vi1Bc31fvj-KKU')

@bot.message_handler(commands=["quote"])
def quote(msg):
    quote = soup.random_quote()
    bot.send_message(
        msg.chat.id, 
        f'*{quote[0]}*\n{quote[1]}',
        parse_mode='markdown',
    )

In [49]:
bot.polling(none_stop=True)